<a href="https://colab.research.google.com/github/anvicoder/Turtle-Trading-Rule/blob/main/Turtle_Trading_Rule.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [124]:
import pandas as pd

df = pd.read_csv("data.csv")

#Check if NA values are in data
df=df[df['volume']!=0]
df.reset_index(drop=True, inplace=True)
df.isna().sum()
print(df)

       datetime      open      high       low     close        volume  \
0    2020-07-16   9190.80   9216.02   9005.00   9130.11   8765.698589   
1    2020-07-17   9130.93   9182.99   9054.99   9154.29   6423.747162   
2    2020-07-18   9154.29   9210.00   9122.65   9175.85   4121.626974   
3    2020-07-19   9175.83   9245.03   9105.37   9212.87   3520.072476   
4    2020-07-20   9214.01   9223.51   9131.00   9161.05   5815.330842   
..          ...       ...       ...       ...       ...           ...   
930  2023-02-01  23127.15  23813.57  22760.00  23735.97  20292.224170   
931  2023-02-02  23735.97  24262.18  23367.87  23493.84  16368.921383   
932  2023-02-03  23497.29  23720.00  23208.01  23433.54  14215.668430   
933  2023-02-04  23433.47  23589.67  23257.71  23328.87   3567.130075   
934  2023-02-05  23329.47  23433.20  22758.01  22936.34   6536.140584   

          reserve  funding_rates      mvrv          nrpl      nupl  \
0    2.889262e+06       0.009332  1.563972 -3.694023e

In [125]:
missing_values = data.isnull().sum()
data.dropna(inplace=True)


In [126]:
print(df.head())

     datetime     open     high      low    close       volume       reserve  \
0  2020-07-16  9190.80  9216.02  9005.00  9130.11  8765.698589  2.889262e+06   
1  2020-07-17  9130.93  9182.99  9054.99  9154.29  6423.747162  2.889943e+06   
2  2020-07-18  9154.29  9210.00  9122.65  9175.85  4121.626974  2.890642e+06   
3  2020-07-19  9175.83  9245.03  9105.37  9212.87  3520.072476  2.892067e+06   
4  2020-07-20  9214.01  9223.51  9131.00  9161.05  5815.330842  2.891858e+06   

   funding_rates      mvrv          nrpl      nupl  stock_to_flow_reversion  \
0       0.009332  1.563972 -3.694023e+06  0.360602                 0.421011   
1       0.007262  1.568555  3.367513e+05  0.362471                 0.366354   
2       0.004125  1.572507  7.637219e+06  0.364073                 0.404649   
3       0.007809  1.579355  2.053120e+07  0.366830                 0.409607   
4       0.010244  1.568483  7.103351e+06  0.362441                 0.437860   

   sth_sopr signal        RSI        9_ema  

In [127]:
print(df.dtypes)

datetime                    object
open                       float64
high                       float64
low                        float64
close                      float64
volume                     float64
reserve                    float64
funding_rates              float64
mvrv                       float64
nrpl                       float64
nupl                       float64
stock_to_flow_reversion    float64
sth_sopr                   float64
signal                      object
RSI                        float64
9_ema                      float64
21_ema                     float64
50_ema                     float64
200_ema                    float64
Fear_and_Greed_Index         int64
dtype: object


In [128]:
#Performing Linear regression because close has continuous data and mostly close is prefeeered in turtle trading rule
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score



# Assuming 'X' contains the features and 'y' contains the target variable
# Replace 'feature_columns' with the columns containing your features
# Replace 'target_column' with the column containing your target variable
X = df[['close']]  # Add more feature columns as needed
y = df['close']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the Linear Regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)


Mean Squared Error: 1.433189938327171e-23
R-squared: 1.0


In [129]:
print(X_train.dtypes)
print(y_train.dtypes)

close    float64
dtype: object
float64


# Performing turtle trading rule

In [123]:
import pandas as pd
import matplotlib.pyplot as plt

def turtle_trading(df, n=20):
    """
    Applies the Turtle Trading Rule to a DataFrame.

    Args:
        df: Pandas DataFrame containing the price data.
        n: Period for moving average and Donchian Channel calculations.

    Returns:
        buy_signals: List of buy signal timestamps.
        sell_signals: List of sell signal timestamps.
    """
    buy_signals, sell_signals = [], []

    # Calculate moving average
    df['moving_average'] = df['close'].rolling(window=n).mean()

    # Calculate Donchian Channel
    df['upper_band'] = df['high'].rolling(window=n).max()
    df['lower_band'] = df['low'].rolling(window=n).min()

    for i in range(1, len(df)):
        current_close = df['close'][i]
        previous_close = df['close'][i-1]
        moving_average = df['moving_average'][i]
        upper_band = df['upper_band'][i]
        lower_band = df['lower_band'][i]

        if current_close > moving_average and previous_close <= moving_average and current_close > upper_band:
            buy_signals.append(df.index[i])
        elif current_close < moving_average and previous_close >= moving_average and current_close < lower_band:
            sell_signals.append(df.index[i])

    return buy_signals, sell_signals



# Convert 'datetime' column to datetime type
df['datetime'] = pd.to_datetime(df['datetime'])

# Call the turtle_trading function to get buy and sell signals
buy_signals, sell_signals = turtle_trading(df)

# Plot the open and close prices along with the buy and sell signals
plt.figure(figsize=(10, 6))
plt.plot(df['datetime'], df['open'], color='blue', label='Open', linestyle='-')
plt.plot(df['datetime'], df['close'], color='red', label='Close', linestyle='-')
plt.scatter(buy_signals, df.loc[buy_signals, 'close'], marker='^', color='green', label='Buy Signal')
plt.scatter(sell_signals, df.loc[sell_signals, 'close'], marker='^', color='red', label='Sell Signal')
plt.xlabel('Date')
plt.ylabel('Price')
plt.title('Turtle Trading Strategy')
plt.legend()
plt.grid(True)
plt.show()


<IPython.core.display.Javascript object>